# 구현 과제
1. 크롤링 기능 하나의 함수로 변환
2. 구, 동의 리스트 제작
3. 반복문 활용하여 모든 구, 동의 단지명을 DB에 저장
4. RAG시스템으로 유효성 검사 구축

In [67]:
%pip install timezone

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement timezone (from versions: none)
ERROR: No matching distribution found for timezone

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timezone
from bs4 import BeautifulSoup
from config import settings
import time
import json
import os
import logging

In [2]:
UTC = timezone.utc

# 로깅 설정
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# 환경 변수
MONGODB_URI = os.getenv("MONGODB_URI", "mongodb://localhost:27017/")
MONGODB_DB = os.getenv("MONGODB_DB", "converter_db")

try:
    client = MongoClient(MONGODB_URI)
    db = client[MONGODB_DB]
    work_client = MongoClient(settings.WORK_MONGODB_URI)
    work_db = work_client[settings.WORK_MONGODB_DB]
    vocab_db = work_client[settings.VOCAB_MONGODB_DB]
    logger.info("MongoDB 연결 성공")
    print("MongoDB 연결 성공")
except Exception as e:
    logger.error(f"MongoDB 연결 실패: {str(e)}")
    raise

MongoDB 연결 성공


In [3]:
chrome_options = Options()

In [5]:
service = Service('./chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

In [6]:
# 부동산 뱅크 로그인 페이지 이동
driver.get('https://www.neonet.co.kr/novo-rebank/view/member/MemberLogin.neo?login_check=yes&return_url=/novo-rebank/index.neo')

In [7]:
# 로그인
id_input = driver.find_element(By.XPATH, '//*[@id="input_id"]')
id_input.send_keys('5164980')
pwd_input = driver.find_element(By.XPATH, '//*[@id="input_pw"]')
pwd_input.send_keys('5164980')
button = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/dl/dt/form/table[1]/tbody/tr[1]/td[2]/input')
button.click()
time.sleep(2)
driver.switch_to.window(driver.window_handles[-1])

In [8]:
# 매물 등록 페이지로 이동
button_2 = driver.find_element(By.XPATH, '//*[@id="btn_offer_register"]/span/a')
button_2.click()
time.sleep(2)

# 아파트랑 오피스텔만 크롤링하면 됨.

In [9]:
# 아파트 단지명 크롤링
property_type = driver.find_element(By.XPATH, '//*[@id="offerings_gbn"]')
property_type_select = Select(property_type)
property_type_select.select_by_value('AT')  # 오피스텔 'op'
time.sleep(1)

# 서울 셀렉
Cities = driver.find_element(By.XPATH, '//*[@id="metro_cd"]')
Cities_select = Select(Cities)
Cities_select.select_by_value('1100000000')
time.sleep(1)

# 구-동-단지 순회
Gus = driver.find_element(By.XPATH, '//*[@id="gu_cd"]')
Gus_select = Select(Gus)

# 구 순회
for gu_option in Gus_select.options[1:]:  # 첫 번째 옵션(선택하세요) 제외
    gu_value = gu_option.get_attribute('value')
    gu_name = gu_option.text
    Gus_select.select_by_value(gu_value)
    time.sleep(1)
    
    # 각 구별 데이터 초기화
    gu_data = {
        '_id': gu_value,  # 구 코드를 id로 사용
        'name': gu_name,
        'created_at': datetime.now(),
        'updated_at': datetime.now(),
        'dongs': {}
    }
    
    # 동 순회
    Dongs = driver.find_element(By.XPATH, '//*[@id="dong_cd"]')
    Dongs_select = Select(Dongs)
    
    for dong_option in Dongs_select.options[1:]:
        dong_value = dong_option.get_attribute('value')
        dong_name = dong_option.text
        Dongs_select.select_by_value(dong_value)
        time.sleep(1)
        
        # 단지명 가져오기
        Danjis = driver.find_element(By.XPATH, '//*[@id="complex_cd"]')
        Danjis_select = Select(Danjis)
        all_Danjis_options = [option.text for option in Danjis_select.options][1:]
        
        # 동별 단지 데이터 저장
        gu_data['dongs'][dong_name] = all_Danjis_options
        
        print(f"크롤링 완료: {gu_name} {dong_name} - {len(all_Danjis_options)}개 단지")

    try:
        # 각 구별로 별도 document로 저장
        vocab_db.seoul.update_one(
            {"_id": gu_value},  # 구 코드로 찾기
            {"$set": gu_data},  # 업데이트할 데이터
            upsert=True  # 없으면 삽입
        )
        print(f"{gu_name} 데이터 저장 완료")
    except Exception as e:
        logger.error(f"데이터 저장 실패: {str(e)}")
        print(f"데이터 저장 실패: {str(e)}")
        raise

크롤링 완료: 강남구 개포동 - 47개 단지
크롤링 완료: 강남구 논현동 - 113개 단지
크롤링 완료: 강남구 대치동 - 74개 단지
크롤링 완료: 강남구 도곡동 - 77개 단지
크롤링 완료: 강남구 삼성동 - 110개 단지
크롤링 완료: 강남구 세곡동 - 7개 단지
크롤링 완료: 강남구 수서동 - 8개 단지
크롤링 완료: 강남구 신사동 - 36개 단지
크롤링 완료: 강남구 압구정동 - 21개 단지
크롤링 완료: 강남구 역삼동 - 156개 단지
크롤링 완료: 강남구 율현동 - 2개 단지
크롤링 완료: 강남구 일원동 - 20개 단지
크롤링 완료: 강남구 자곡동 - 8개 단지
크롤링 완료: 강남구 청담동 - 123개 단지
강남구 데이터 저장 완료
크롤링 완료: 강동구 강일동 - 15개 단지
크롤링 완료: 강동구 고덕동 - 11개 단지
크롤링 완료: 강동구 길동 - 129개 단지
크롤링 완료: 강동구 둔촌동 - 61개 단지
크롤링 완료: 강동구 명일동 - 41개 단지
크롤링 완료: 강동구 상일동 - 20개 단지
크롤링 완료: 강동구 성내동 - 163개 단지
크롤링 완료: 강동구 암사동 - 67개 단지
크롤링 완료: 강동구 천호동 - 170개 단지
강동구 데이터 저장 완료
크롤링 완료: 강북구 미아동 - 44개 단지
크롤링 완료: 강북구 번동 - 37개 단지
크롤링 완료: 강북구 수유동 - 82개 단지
크롤링 완료: 강북구 우이동 - 10개 단지
강북구 데이터 저장 완료
크롤링 완료: 강서구 가양동 - 33개 단지
크롤링 완료: 강서구 공항동 - 36개 단지
크롤링 완료: 강서구 내발산동 - 103개 단지
크롤링 완료: 강서구 등촌동 - 106개 단지
크롤링 완료: 강서구 마곡동 - 44개 단지
크롤링 완료: 강서구 방화동 - 96개 단지
크롤링 완료: 강서구 염창동 - 87개 단지
크롤링 완료: 강서구 화곡동 - 429개 단지
강서구 데이터 저장 완료
크롤링 완료: 관악구 남현동 - 59개 단지
크롤링 완료: 관악구 봉천동 - 204개 단지
크롤링 완료: 관악구 